# PDF document text embeddings generation and storage in a Milvus Vector DB example notebook

Prerequisites:
- need a Milvus instance
- configure the Milvus credentials in the Workbench environment using the RHOAI dashboard

### Install required library depencencies

In [1]:
!pip install -q einops==0.7.0 langchain==0.1.9 pypdf==4.0.2 pymilvus==2.3.6 sentence-transformers==2.4.0


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import requests
import os
from langchain.document_loaders import PyPDFDirectoryLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus

In [3]:
# Replace values according to your Milvus deployment
MILVUS_HOST = "vectordb-milvus.milvus.svc.cluster.local"
MILVUS_PORT = 19530
MILVUS_USERNAME = os.getenv('MILVUS_USERNAME')
MILVUS_PASSWORD = os.getenv('MILVUS_PASSWORD')
MILVUS_COLLECTION = "demo_collection"

#### As example we shall index the Red Hat OpneShift AI (RHOAI) docs, for a given release (e.g. 2.6)

In [4]:
product_version = 2.6
documents = [
    "release_notes",
    "introduction_to_red_hat_openshift_ai",
    "getting_started_with_red_hat_openshift_ai_self-managed",
    "openshift_ai_tutorial_-_fraud_detection_example",
    "developing_a_model",
    "integrating_data_from_amazon_s3",
    "working_on_data_science_projects",
    "serving_models",
    "monitoring_data_science_models",
    "managing_users",
    "managing_resources",
    "installing_and_uninstalling_openshift_ai_self-managed",
    "installing_and_uninstalling_openshift_ai_self-managed_in_a_disconnected_environment",
    "upgrading_openshift_ai_self-managed",
    "upgrading_openshift_ai_self-managed_in_a_disconnected_environment",   
]

pdfs = [f"https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/{product_version}/pdf/{doc}/red_hat_openshift_ai_self-managed-{product_version}-{doc}-en-us.pdf" for doc in documents]
pdfs_to_urls = {f"red_hat_openshift_ai_self-managed-{product_version}-{doc}-en-us": f"https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/{product_version}/html-single/{doc}/index" for doc in documents}

In [5]:
os.mkdir(f"rhoai-doc-{product_version}")

for pdf in pdfs:
    try:
        response = requests.get(pdf)
    except:
        print(f"Skipped {pdf}")
        continue
    if response.status_code!=200:
        print(f"Skipped {pdf}")
        continue  
    with open(f"rhoai-doc-{product_version}/{pdf.split('/')[-1]}", 'wb') as f:
        f.write(response.content)

Skipped https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/2.6/pdf/monitoring_data_science_models/red_hat_openshift_ai_self-managed-2.6-monitoring_data_science_models-en-us.pdf


In [6]:
pdf_folder_path = f"./rhoai-doc-{product_version}"

pdf_loader = PyPDFDirectoryLoader(pdf_folder_path)
pdf_docs = pdf_loader.load()

In [7]:
from pathlib import Path

for doc in pdf_docs:
    doc.metadata["source"] = pdfs_to_urls[Path(doc.metadata["source"]).stem]

#### We also use the website information in addition to the PDF files

In [8]:
websites = [
    "https://ai-on-openshift.io/getting-started/openshift/",
    "https://ai-on-openshift.io/getting-started/opendatahub/",
    "https://ai-on-openshift.io/getting-started/openshift-ai/",
    "https://ai-on-openshift.io/odh-rhoai/configuration/",
    "https://ai-on-openshift.io/odh-rhoai/custom-notebooks/",
    "https://ai-on-openshift.io/odh-rhoai/nvidia-gpus/",
    "https://ai-on-openshift.io/odh-rhoai/custom-runtime-triton/",
    "https://ai-on-openshift.io/odh-rhoai/openshift-group-management/",
    "https://ai-on-openshift.io/tools-and-applications/minio/minio/",
    "https://access.redhat.com/articles/7047935",
    "https://access.redhat.com/articles/rhoai-supported-configs",
]

In [9]:
website_loader = WebBaseLoader(websites)
website_docs = website_loader.load()

#### We merge the docs and website info then we split the results into document chunks with some overlap

In [10]:
docs = pdf_docs + website_docs

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024,
                                               chunk_overlap=128)
all_splits = text_splitter.split_documents(docs)
all_splits[0]

Document(page_content='Red Hat OpenShift AI Self-Managed\n \n2.6\nDeveloping a model\nUnderstand the development and deployment workflow and deploy your models in\nintelligent applications\nLast Updated: 2024-03-26', metadata={'source': 'https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/2.6/html-single/developing_a_model/index', 'page': 0})

#### Create the index and ingest the documents in Milvus.
Note that the results will appear in Milvus only after the add_documents operation has completed

In [12]:
# If you want to use a GPU, change the model_kwargs as below, otherwise leave it empty, as in the default setting
#model_kwargs = {'device': 'cuda'}
model_kwargs = {}
embeddings = HuggingFaceEmbeddings(
    model_kwargs=model_kwargs,
    show_progress=True
)

# BEWARE: `drop_old` is set to True, so if the collection already existed it will deleted first.
db = Milvus(
    embedding_function=embeddings,
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT, "user": MILVUS_USERNAME, "password": MILVUS_PASSWORD},
    collection_name=MILVUS_COLLECTION,
    metadata_field="metadata",
    text_field="page_content",
    auto_id=True,
    drop_old=True
    )

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
db.add_documents(all_splits)

Batches:   0%|          | 0/35 [00:00<?, ?it/s]

[448694912970926986,
 448694912970926987,
 448694912970926988,
 448694912970926989,
 448694912970926990,
 448694912970926991,
 448694912970926992,
 448694912970926993,
 448694912970926994,
 448694912970926995,
 448694912970926996,
 448694912970926997,
 448694912970926998,
 448694912970926999,
 448694912970927000,
 448694912970927001,
 448694912970927002,
 448694912970927003,
 448694912970927004,
 448694912970927005,
 448694912970927006,
 448694912970927007,
 448694912970927008,
 448694912970927009,
 448694912970927010,
 448694912970927011,
 448694912970927012,
 448694912970927013,
 448694912970927014,
 448694912970927015,
 448694912970927016,
 448694912970927017,
 448694912970927018,
 448694912970927019,
 448694912970927020,
 448694912970927021,
 448694912970927022,
 448694912970927023,
 448694912970927024,
 448694912970927025,
 448694912970927026,
 448694912970927027,
 448694912970927028,
 448694912970927029,
 448694912970927030,
 448694912970927031,
 448694912970927032,
 448694912970

#### Run a test query

In [14]:
query = "How can I work with GPU and taints in OpenShift AI?"
docs_with_score = db.similarity_search_with_score(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.5976204872131348
Apply the taints you need to your Nodes or MachineSets, for example:
apiVersion: machine.openshift.io/v1beta1
kind: MachineSet
metadata:
  ...
spec:
  replicas: 1
  selector:
    ...
  template:
    ...
    spec:
      ...
      taints:
        - key: restrictedaccess
          value: "yes"
          effect: NoSchedule



Apply the relevant toleration to the NVIDIA Operator.


In the nvidia-gpu-operator namespace, get to the Installed Operator menu, open the NVIDIA GPU Operator settings, get to the ClusterPolicy tab, and edit the ClusterPolicy.
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Score:  0.6429101228713989
CHAPTER 9. ENABLING GPU SUPPORT IN OPENSHIFT AI
Optionally, to ensure that your data scientists can use compute-heavy workloads in their models, you
can